# [Research Request - Explore data for PUC 99314.11 leg report #1613](https://github.com/cal-itp/data-analyses/issues/1613)

Discussed with Peter and Cayman about PUC 99314.11, specifically about sub-section (d)(1)

>(d) (1) On or before November 30, 2025, the department shall submit a report to the Legislature on the revenue vehicle hours, ridership, and passenger mile impacts on the services offered by operators to which Sections 99314.6 and 99314.7 do not apply pursuant to subdivision (a).

1. read in SCO data
2. read in warehouse data for annual ntd reporters

Compare both sources, see which agencies join or not join in either list. 

Compare data in both sources, see if any annual totals match

In [1]:
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
import altair as alt

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Read in SCO data
Data is speard out among different excel files for different state fiscal years

In [ ]:
url_23_24 = "https://bythenumbers.sco.ca.gov/download/bdqr-pszz/application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"
url_21_22 = "https://bythenumbers.sco.ca.gov/download/aqpg-as24/application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"
url_18_20 = "https://bythenumbers.sco.ca.gov/download/6dj3-r4jw/application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"

all_years = [
    url_23_24,
    url_21_22,
    url_18_20   
]

In [ ]:
# what are the sheet names for each excel file?
for data in all_years:
    sco_info = pd.ExcelFile(data)
    print(sco_info.sheet_names)

In [ ]:
def make_sco_df(url:str, sheet:str) -> pd.DataFrame:
    """
    Function that reads in each excel url at specific sheet name. then snakecases all the columns
    """
    df = pd.read_excel(url, sheet_name = sheet)
    df = to_snakecase(df)
    
    return df

In [ ]:
# make DFs for each excel sheet
ops_18_20 = make_sco_df(url_18_20, "TO_OPERATING_DATA")
ops_21_22 = make_sco_df(url_21_22, "5 TO_OPERATING_DATA")
ops_23_24 = make_sco_df(url_23_24, "5 TO_OPERATING_DATA")

In [ ]:
# examine each sheet was read in successfully
display(
    ops_18_20.shape,
    ops_21_22.shape,
    ops_23_24.shape
)

## SCO data cleaning

In [ ]:
# do all DFs have the same columns?
set(ops_18_20.columns) == set(ops_21_22.columns) == set(ops_23_24.columns) #TRUE!

In [ ]:
# group column names by similar categories 

date_cols =[
    'date_service_began_operations__mm_dd_yyyy__motor_bus',
    'date_service_began_operations__mm_dd_yyyy__heavy_rail',
    'date_service_began_operations__mm_dd_yyyy__light_rail',
    'date_service_began_operations__mm_dd_yyyy__trolley_bus',
    'date_service_began_operations__mm_dd_yyyy__ferry_boat',
    'date_service_began_operations__mm_dd_yyyy__demand_response_vehicles',
    'date_service_began_operations__mm_dd_yyyy__vanpool',
    'date_service_began_operations__mm_dd_yyyy__other_transportation_mode__specify_'
]

vrh_cols = [
    "total_actual_vehicle_revenue_hours_—_annual_demand_response_vehicles_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_ferry_boat_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_heavy_rail_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_light_rail_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_motor_bus_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_other_transportation_mode__specify__actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_trolley_bus_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_vanpool_actual_vehicle_revenue_hours_—_annual",
]

vrm_cols = [
    "total_actual_vehicle_revenue_miles_demand_response_vehicles",
    "total_actual_vehicle_revenue_miles_ferry_boat",
    "total_actual_vehicle_revenue_miles_heavy_rail",
    "total_actual_vehicle_revenue_miles_light_rail",
    "total_actual_vehicle_revenue_miles_motor_bus",
    "total_actual_vehicle_revenue_miles_other_transportation_mode__specify_",
    "total_actual_vehicle_revenue_miles_trolley_bus",
    "total_actual_vehicle_revenue_miles_vanpool",
]

passenger_cols = [
    "total_passengers_—_annual_demand_response_vehicles",
    "total_passengers_—_annual_ferry_boat",
    "total_passengers_—_annual_heavy_rail",
    "total_passengers_—_annual_light_rail",
    "total_passengers_—_annual_motor_bus",
    "total_passengers_—_annual_other_transportation_mode__specify_",
    "total_passengers_—_annual_trolley_bus",
    "total_passengers_—_annual_vanpool",
]

group_list = ['entity_name', 'fiscal_year', 'entity_id']

In [ ]:
# do all DFs have the same column datatypes?
display(
    ops_18_20.dtypes.equals(ops_21_22.dtypes),
    ops_18_20.dtypes.equals(ops_23_24.dtypes),
    ops_21_22.dtypes.equals(ops_23_24.dtypes),
)

# op_18_20 data types dont match. 


In [ ]:
# columns dont match?
col_cats ={
    "date columns":date_cols,
    "vrh columns":vrh_cols,
    "vrm columns":vrm_cols,
    "passenger columns":passenger_cols
}

for name, cols in col_cats.items():
    print(f"""Check if data types in {name} match:
    FY 18-20 vs 21-22: {ops_18_20[cols].dtypes.equals(ops_21_22[cols].dtypes)},
    FY 18-20 vs 23-24: {ops_18_20[cols].dtypes.equals(ops_23_24[cols].dtypes)},
    """)
    

## cleaning date time columns

In [ ]:
all_ops = [
    ops_18_20,
    ops_21_22,
    ops_23_24
]

# what do the date column data types look like in each df?
for df in all_ops:
    display(
        df[date_cols].info()
    )
    
# ops_18_20.date_service_began_operations__mm_dd_yyyy__demand_response_vehicles is an object, everything else is datetime64[ns]

In [ ]:
# attempt to change data type from object to datetime64[ns]?
ops_18_20[
    "date_service_began_operations__mm_dd_yyyy__demand_response_vehicles"] = pd.to_datetime(ops_18_20[
    "date_service_began_operations__mm_dd_yyyy__demand_response_vehicles"], errors= "coerce" )

# OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2975-04-21 00:00:00 present at position 141. need errors= "coerce" to make invalid datetimes be NaT

In [ ]:
# connfrming all datatypes match
display(
    ops_18_20.dtypes.equals(ops_21_22.dtypes),
    ops_18_20.dtypes.equals(ops_23_24.dtypes),
    ops_21_22.dtypes.equals(ops_23_24.dtypes),
) # TRUE!

In [ ]:
# check years in each dataframe
for df in all_ops:
    print(df["fiscal_year"].value_counts())

## Combining all datasets 

In [ ]:
# Since all DFs have the same columns, concat all DFs together to 1 big raw DF
ops_18_24 = pd.concat(all_ops, ignore_index = True)

In [ ]:
display(
    type(ops_18_24),
    ops_18_24["fiscal_year"].value_counts(),
    ops_18_24.shape,
    list(ops_18_24.columns)
)



## Sum common metrics columns, remove unused columns

In [ ]:
ops_18_24["total_upt"] = ops_18_24[passenger_cols].sum(axis=1)
ops_18_24["total_vrh"] = ops_18_24[vrh_cols].sum(axis=1)
ops_18_24["total_vrm"] = ops_18_24[vrm_cols].sum(axis=1)

In [ ]:
# keep specific columns
sco_18_24 = ops_18_24[group_list+["total_upt","total_vrh","total_vrm"]]

In [ ]:
sco_18_24.info()

## Saving data to GCS as parquet

In [ ]:
# saving DFs to GCS

gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
all_sco_op_data = "sco_operator_data_18_24.parquet"
filtered_sco_data = "sco_upt_vrm_pmt_18_24.parquet"

# saving as parquet
# ops_18_24.to_parquet(f"{gcs_path}{all_sco_op_data}")
# sco_18_24.to_parquet(f"{gcs_path}{filtered_sco_data}")

# saving as csv
# ops_18_24.to_csv(f"{gcs_path}sco_operator_data_18_24.csv")
# sco_18_24.to_csv(f"{gcs_path}sco_upt_vrm_pmt_18_24.csv")

## Reading in data from GCS as parquet

In [ ]:
# read in parquet, ensure it works 
ops_18_24 = pd.read_parquet(f"{gcs_path}{all_sco_op_data}")

sco_18_24 = pd.read_parquet(f"{gcs_path}{filtered_sco_data}")

In [ ]:
ops_18_24.info(),
sco_18_24.info()

## SCO Summary findings

In [ ]:
# How many unique entities are there 
sco_18_24["entity_name"].nunique()

In [ ]:
# how many unique entiries are there from each FY
sco_18_24.groupby("fiscal_year")["entity_name"].nunique()

In [ ]:
# how many unique entity id are there?
sco_18_24["entity_id"].nunique()

In [ ]:
# how many have "specialized services" in their name
sco_18_24[sco_18_24["entity_name"].str.contains("Specialized Service")]["entity_name"].nunique()

In [ ]:
# how many entities do not have "specialized service"
sco_18_24[~sco_18_24["entity_name"].str.contains("Specialized Service")]["entity_name"].nunique()

# Read NTD data from warehouse


New syntax to query the warehouse
```
from calitp_data_analysis.sql import get_engine

db_engine = get_engine()

with db_engine.connect() as connection:
    query = ""
    df = pd.read_sql(query, connection)
```

In [2]:
from calitp_data_analysis.sql import get_engine

In [4]:
db_engine = get_engine()

metric_list = [
    "pmt",
    "upt",
    "vrh"
]

# empty list for appending DFs
df_list = []
with db_engine.connect() as connection:
    for metric in metric_list:
        query = f"""
        SELECT
          ntd_id,
          SUM({metric}) AS total_{metric},
          source_agency,
          agency_status,
          reporter_type,
          year
        FROM
          `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
        WHERE
          source_state = "CA"
          AND year >= 2018
        GROUP BY
          ntd_id,
          source_agency,
          agency_status,
          reporter_type,
          year
        """
        # create df
        metric = pd.read_sql(query,connection) 
        
        # append df to list
        df_list.append(metric)
        

In [5]:
# confirm DFs were populated in list
display(
    len(df_list),
    type(df_list[0]),type(df_list[1]),type(df_list[2]),
    df_list[0].columns, df_list[1].columns, df_list[2].columns
)

3

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

Index(['ntd_id', 'total_pmt', 'source_agency', 'agency_status',
       'reporter_type', 'year'],
      dtype='object')

Index(['ntd_id', 'total_upt', 'source_agency', 'agency_status',
       'reporter_type', 'year'],
      dtype='object')

Index(['ntd_id', 'total_vrh', 'source_agency', 'agency_status',
       'reporter_type', 'year'],
      dtype='object')

In [6]:
# unpack list into separate DFs
ntd_pmt, ntd_upt, ntd_vrh = df_list

In [7]:
# check if unpack was successful
ntd_pmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ntd_id         1524 non-null   object 
 1   total_pmt      1291 non-null   float64
 2   source_agency  1614 non-null   object 
 3   agency_status  1614 non-null   object 
 4   reporter_type  1614 non-null   object 
 5   year           1614 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 75.8+ KB


In [8]:
# confirm the same years exist in all DFs
display(
    set(ntd_pmt["year"].sort_values().unique()) == set(ntd_upt["year"].sort_values().unique()) == set(ntd_vrh["year"].sort_values().unique())
)

True

# Merge data frames together

In [9]:
merge_cols =["ntd_id", "year","source_agency","agency_status", "reporter_type"]

merge_1 = ntd_pmt.merge(ntd_upt, on= merge_cols, how = "inner")

In [10]:
ntd_all_metrics = merge_1.merge(ntd_vrh, on= merge_cols, how = "inner")

In [11]:
col_order =[
    "ntd_id",
    "source_agency",
    "agency_status",
    "reporter_type",
    "year",
    "total_upt",
    "total_vrh",
    "total_pmt"
]

ntd_all_metrics = ntd_all_metrics[col_order]

In [12]:
ntd_all_metrics.head(10)

,ntd_id,source_agency,agency_status,reporter_type,year,total_upt,total_vrh,total_pmt
0,90198,City of Porterville (COLT) - Transit Department,Active,Building Reporter,2018,648649.0,52799.0,0.0
1,90198,City of Porterville (COLT) - Transit Department,Active,Building Reporter,2022,260879.0,41460.0,0.0
2,90198,City of Porterville (COLT) - Transit Department,Active,Building Reporter,2020,522056.0,47356.0,0.0
3,90198,City of Porterville (COLT) - Transit Department,Active,Building Reporter,2023,NaN,NaN,NaN
4,90198,City of Porterville (COLT) - Transit Department,Active,Building Reporter,2019,635559.0,52834.0,0.0
5,90198,City of Porterville (COLT) - Transit Department,Active,Building Reporter,2021,174595.0,30773.0,0.0
6,90036,Orange County Transportation Authority (OCTA),Active,Full Reporter,2023,32674688.0,2076903.0,137085597.0
7,90036,Orange County Transportation Authority (OCTA),Active,Full Reporter,2022,27753507.0,1897023.0,132745034.0
8,90036,Orange County Transportation Authority (OCTA),Active,Full Reporter,2019,40743654.0,2638126.0,203590805.0
9,90036,Orange County Transportation Authority (OCTA),Active,Full Reporter,2020,33009047.0,2232009.0,155734272.0


## saving data to gcs

In [13]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
ntd_name = "ntd_operator_data_18_23.parquet"

# ntd_all_metrics.to_parquet(f"{gcs_path}{ntd_name}")
# ntd_all_metrics.to_csv(f"{gcs_path}ntd_operator_data_18_23.csv")

In [14]:
ntd_all_metrics = pd.read_parquet(f"{gcs_path}{ntd_name}")

In [15]:
ntd_all_metrics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1614 entries, 0 to 1613
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ntd_id         1524 non-null   object 
 1   source_agency  1614 non-null   object 
 2   agency_status  1614 non-null   object 
 3   reporter_type  1614 non-null   object 
 4   year           1614 non-null   int64  
 5   total_upt      1291 non-null   float64
 6   total_vrh      1291 non-null   float64
 7   total_pmt      1291 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 113.5+ KB


# compare sco data to ntd data

## Summary 

In [ ]:
print(f"""Unique agencies in SCO data: {len(sco_18_24["entity_name"].unique())},
How many SCO agencies are "Specialized Services": {len(sco_18_24[sco_18_24["entity_name"].str.contains("Specialized Service")]["entity_name"].unique())}
    
Unique agencies in NTD data: {len(ntd_all_metrics["source_agency"].unique())},
""")

## Who are the common agencies from each list?

SCO list had short entity names. The short names were used to match with the longer names in ntd data

In [ ]:
sco_agencies = list(sco_18_24["entity_name"].unique())

pattern = "|".join(sco_agencies)

In [ ]:
print(f"""Number of fuzzy match of names between sco and NTD list:
    {len(ntd_all_metrics[ntd_all_metrics["source_agency"].str.contains(pattern,case=False)]["source_agency"].unique())}
""")

ntd_all_metrics[ntd_all_metrics["source_agency"].str.contains(pattern,case=False)]["source_agency"].unique()

## Who are the unique agencies in each list?

In [ ]:
# **This isnt reliable. found some instances of agencies names appearing in both list
# Glendora appears in both list, 
print(f"""Number of names that dont fuzzy match:
    {len(ntd_all_metrics[~ntd_all_metrics["source_agency"].str.contains(pattern,case=False)]["source_agency"].unique())}
""")

ntd_all_metrics[~ntd_all_metrics["source_agency"].str.contains(pattern,case=False)]["source_agency"].unique()

In [ ]:
# sanity check
agency = "San Luis Obispo"

display(
    ntd_all_metrics[ntd_all_metrics["source_agency"].str.contains(agency)].sort_values(by=["source_agency","year"]).drop(columns="total_pmt"),
    sco_18_24[sco_18_24["entity_name"].str.contains(agency)].sort_values(by=["entity_name","fiscal_year"]).drop(columns="total_vrm")
)
# the SacRT totals match!, Glendora totals matches

# Merced JPA: if you add SCO merced jpa and merced jpa special service, they match the ntd merced jpa numbers
# same as Gardena

# Yosemite Area Regional Transportation System (YARTS) matches in some years and not match in other years 

## test aggregations

In [ ]:
# melt big DF so all columns are under 1 column.
group_list = ['entity_name', 'fiscal_year', 'entity_id']

melt = pd.melt(
    sco_18_24,
    id_vars= group_list,
    # value_vars = ,
    var_name="metric",
    value_name="metric_unit",
    ignore_index=True,
)

In [ ]:
# inspect melted DF 
display(
    melt.info(),
    melt["fiscal_year"].value_counts(),
    melt["metric"].value_counts(),
    melt["entity_name"].value_counts(),
    melt.head(),
)

In [ ]:
# filter melted df by column categories
melt[melt["metric"]=="total_vrm"].info()

In [ ]:
# What does group/agg the melted DF look like?
vrh_total = (
    melt[melt["metric"]=="total_vrh"]
    .groupby(["entity_name","entity_id", "fiscal_year"])["metric_unit"]
    .sum()
    .reset_index()
).rename(columns={"metric_unit":"total_vrh"})

vrm_total = (
    melt[melt["metric"]=="total_vrm"]
    .groupby(["entity_name","entity_id", "fiscal_year"])["metric_unit"]
    .sum()
    .reset_index()
).rename(columns={"metric_unit":"total_vrm"})

passenger_total =(
    melt[melt["metric"]=="total_upt"]
    .groupby(["entity_name","entity_id", "fiscal_year"])["metric_unit"]
    .sum()
    .reset_index()
).rename(columns={"metric_unit":"total_upt"})


In [ ]:
# inspect melted DF by just VRH columns in Sacramento
all_totals = [
    vrh_total,
    vrm_total,
    passenger_total
]

for df in all_totals:
    display(df[df["entity_name"].str.contains("Sacramento")].head()) # some operators do not have data for some modes. this makes sense.

In [ ]:
# inspect melted DF by just VRH columns in Sacramento
for df in all_totals:
    display(df[df["entity_name"].str.contains("Albany - Specialized Service")].head())

In [ ]:
alt.Chart(vrh_total).mark_line(point=True).encode(
    x="fiscal_year:N",
    y="total_vrh:Q",
    color="entity_name:N"
).properties(width="container")